In [1]:
#Imports
import time
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
import urllib.request
from urllib.request import urlopen
import pyautogui
from requests.exceptions import ConnectionError
# from pywebcopy import save_webpage

In [2]:

internal_urls = set()
external_urls = set()

In [3]:
def is_url_valid(url):
   #Checks if the url is valid or not
    parsedURL = urlparse(url)
    return bool(parsedURL.netloc) and bool(parsedURL.scheme)

#The above function works as follows. We check if the scheme is present and there is a value in the network location part
# url = "https://umd.edu/virusinfo"
# urlparse(url)
# ParseResult(scheme='https', netloc='umd.edu', path='/virusinfo', 
# params='', query='', fragment='')

In [4]:

#The function gives all urls
def get_all_urls(url):
    urls = set()
    # domain name of the URL without the protocol (umd.edu in this case)
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            #href is empty and we don't need that a element
            continue
        #if the link is not absolute, make it by joining relative to the base
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        #constructing an absolute URL from parsed data
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_url_valid(href):
            #in valid url
            continue
        if href in internal_urls:
            #it is already in the set, so we don't need to add
            continue
        if domain_name not in href:
            #it is an external link. i.e
            # Check if it is already there 
            if href not in external_urls:
#                 print(f"[EXT] External link: {href}")
                external_urls.add(href)
            continue
#         print(".",end=" ")
        print(f"[INT] Internal link: {href}")

        urls.add(href)
        internal_urls.add(href)
    return urls


In [5]:
total_urls_visited = 0
def crawl(url, max_urls=50):
    #Max URL is just to decrease the time if there are a lot of pages.
    #The following code was openly available of github and I found this
    #idea useful to inhibit crawling time
    global total_urls_visited
    total_urls_visited += 1
    try:
        links = get_all_urls(url)
        for link in links:
            if total_urls_visited > max_urls:
                break
            crawl(link, max_urls=max_urls)
    except:
        print("caught")


In [6]:
#base_url = "https://umd.edu/virusinfo"
#Input
# base_url = input("Enter the URL : ")
base_url = "https://www.umsl.edu/hcdas/"
internal_urls.add(base_url)

In [7]:
parsedurl = urlparse(base_url)
print(parsedurl)
base_url_main = parsedurl.scheme+"://"+parsedurl.netloc
print(base_url_main)
# base_url_text = base_url.split("//",1)[1]
base_url_text = parsedurl.netloc+parsedurl.path
print(base_url_text)
# base_url_text_domain = base_url_text.split("/",1)[0]
base_url_text_domain = parsedurl.netloc
print(base_url_text_domain)

ParseResult(scheme='https', netloc='www.umsl.edu', path='/hcdas/', params='', query='', fragment='')
https://www.umsl.edu
www.umsl.edu/hcdas/
www.umsl.edu


In [8]:
# internal_urls

In [9]:
crawl(base_url)
print("[+] Total External links:", len(external_urls))
print("[+] Total Internal links:", len(internal_urls))
print("[+] Total:", len(external_urls) + len(internal_urls))

[INT] Internal link: https://www.umsl.edu
[INT] Internal link: https://www.umsl.edu/hcdas/index.html
[INT] Internal link: https://www.umsl.edu/admissions/applications.html
[INT] Internal link: https://www.umsl.edu/hcdas/health/index.html
[INT] Internal link: https://www.umsl.edu/hcdas/health/services/index.html
[INT] Internal link: https://www.umsl.edu/hcdas/health/appointments/index.html
[INT] Internal link: https://www.umsl.edu/hcdas/health/immunizations/index.html
[INT] Internal link: https://www.umsl.edu/hcdas/health/info/index.html
[INT] Internal link: https://www.umsl.edu/hcdas/health/forms-policies/index.html
[INT] Internal link: https://www.umsl.edu/hcdas/health/bodyu.html
[INT] Internal link: https://www.umsl.edu/hcdas/health/staff.html
[INT] Internal link: https://www.umsl.edu/hcdas/counseling/index.html
[INT] Internal link: https://www.umsl.edu/hcdas/counseling/visit/index.html
[INT] Internal link: https://www.umsl.edu/hcdas/counseling/services/index.html
[INT] Internal link

[INT] Internal link: https://www.umsl.edu/~webdev/counser/counseling/tips-social-distancing-quarantine-isolation-031620-1.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[INT] Internal link: https://www.umsl.edu/~webdev/counser/health/services/healthsvs.html
[INT] Internal link: https://www.umsl.edu/~webdev/counser/health/services/sexualhealth.html
[INT] Internal link: https://www.umsl.edu/~webdev/counser/health/services/pelvicexams.html
[INT] Internal link: https://www.umsl.edu/hcdas/health/services/smokingcessation.html
[INT] Internal link: https://www.umsl.edu/~webdev/counser/health/services/www.quitterscircle.com
[INT] Internal link: http://www.umsl.edu/services/cps/
[INT] Internal link: https://www.umsl.edu/~webdev/counser/counseling/services/outreach.html
[INT] Internal link: https://www.umsl.edu/~webdev/counser/counseling/services/screenings.html
[INT] Internal link: https://www.umsl.edu/~webdev/counser/counseling/services/consultation.html
[INT] Internal link: https://www.umsl.edu/~webdev/counser/counseling/services/training.html
[INT] Internal link: https://www.umsl.edu/~webdev/counser/counseling/services/groups.html
[INT] Internal link: https

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


...

[INT] Internal link: http://www.umsl.edu/services/academic/strategic-plan/
[INT] Internal link: http://www.umsl.edu/services/academic/index.html
[INT] Internal link: https://www.umsl.edu/services/academic/strategic-plan/index.html
[INT] Internal link: https://www.umsl.edu/services/academic/strategic-plan/student.html
[INT] Internal link: https://www.umsl.edu/services/academic/strategic-plan/research.html
[INT] Internal link: https://www.umsl.edu/services/academic/strategic-plan/community.html
[INT] Internal link: https://www.umsl.edu/services/academic/strategic-plan/inclusion.html
[INT] Internal link: https://www.umsl.edu/services/academic/strategic-plan/planning.html
[INT] Internal link: https://www.umsl.edu/services/academic/strategic-plan/mission.html
[INT] Internal link: https://www.umsl.edu/services/academic/strategic-plan/community-campus.html
[INT] Internal link: https://www.umsl.edu/services/academic/strategic-plan/committee.html
[INT] Internal link: https://www.umsl.edu/servic

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[INT] Internal link: http://www.umsl.edu/services/ctl/about/index.html
[INT] Internal link: http://www.umsl.edu/services/ctl/about/staff.html
[INT] Internal link: http://www.umsl.edu/services/ctl/about/partners.html
[INT] Internal link: http://www.umsl.edu/services/ctl/about/summary.html
[INT] Internal link: http://www.umsl.edu/services/ctl/about/advisory_group.html
[INT] Internal link: http://www.umsl.edu/services/ctl/faculty/index.html
[INT] Internal link: http://www.umsl.edu/services/ctl/faculty/facultyorientations/policies_and_procedures.html
[INT] Internal link: http://www.umsl.edu/services/ctl/faculty/facultyorientations/index.html
[INT] Internal link: http://www.umsl.edu/services/ctl/faculty/facultycolloquia/index.html
[INT] Internal link: http://www.umsl.edu/services/ctl/faculty/CALF/index.html
[INT] Internal link: http://www.umsl.edu/services/ctl/faculty/servicelearning/index.html
[INT] Internal link: http://www.umsl.edu/services/ctl/faculty/instructionalsupport/index.html
[IN

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[+] Total External links: 183
[+] Total Internal links: 373
[+] Total: 556


In [ ]:
count = 0
last_count = 989874
immediate_urls = set() #Linked Associated with the current page, https://umd.edu/virusinfo/ in this case

for i in range(0,4):
    for url in internal_urls:
        if base_url_text in url:
            immediate_urls.add(url)
    count = len(immediate_urls)
    print(count,i)
    if(count == last_count):
        break
    last_count = count
    for immediate_url in immediate_urls:
        try:
            crawl(immediate_url)
            print("[+] NEW Total External links:", len(external_urls))
            print("[+] NEW Total Internal links:", len(internal_urls))
            print("[+] NEW Total:", len(external_urls) + len(internal_urls))
            if len(internal_urls)>1000:
                break
        except requests.exceptions.ConnectionError:
            continue

41 0
[+] NEW Total External links: 183
[+] NEW Total Internal links: 373
[+] NEW Total: 556
[INT] Internal link: http://www.umsl.edu/maps.html
[+] NEW Total External links: 183
[+] NEW Total Internal links: 374
[+] NEW Total: 557
[INT] Internal link: https://www.umsl.edu/hcdas/health/forms-policies/apptforms.html
[INT] Internal link: https://www.umsl.edu/hcdas/health/forms-policies/recordsrelease.html
[INT] Internal link: https://www.umsl.edu/hcdas/health/forms-policies/classexcuse.html
[INT] Internal link: https://www.umsl.edu/hcdas/health/forms-policies/confidentiality.html
[+] NEW Total External links: 184
[+] NEW Total Internal links: 378
[+] NEW Total: 562
[INT] Internal link: https://www.umsl.edu/hcdas/health/info/insfees.html
[INT] Internal link: https://www.umsl.edu/hcdas/health/info/faqs.html
[+] NEW Total External links: 186
[+] NEW Total Internal links: 380
[+] NEW Total: 566
[INT] Internal link: https://www.umsl.edu/hcdas/counseling/faculty-and-staff/trainingsandworkshops.h

[+] NEW Total External links: 202
[+] NEW Total Internal links: 447
[+] NEW Total: 649
[+] NEW Total External links: 203
[+] NEW Total Internal links: 447
[+] NEW Total: 650
[+] NEW Total External links: 203
[+] NEW Total Internal links: 447
[+] NEW Total: 650
[INT] Internal link: https://www.umsl.edu/hcdas/das/Faculty%20Resources/faqs.html
[INT] Internal link: https://www.umsl.edu/hcdas/das/Faculty%20Resources/syllabus.html
[INT] Internal link: https://www.umsl.edu/hcdas/das/Faculty%20Resources/volunteernotetaker.html
[+] NEW Total External links: 203
[+] NEW Total Internal links: 450
[+] NEW Total: 653
[INT] Internal link: https://www.umsl.edu/hcdas/counseling/notification-3-23-20.html
[INT] Internal link: https://www.umsl.edu/hcdas/counseling/howdoiknow.html
[INT] Internal link: https://www.umsl.edu/hcdas/counseling/guide.html
[INT] Internal link: https://www.umsl.edu/hcdas/counseling/screenings.html
[+] NEW Total External links: 203
[+] NEW Total Internal links: 454
[+] NEW Total: 

[+] NEW Total External links: 347
[+] NEW Total Internal links: 505
[+] NEW Total: 852
[+] NEW Total External links: 347
[+] NEW Total Internal links: 505
[+] NEW Total: 852
[INT] Internal link: https://www.umsl.edu/hcdas/_internal/old/coun-full/About Counseling/privacy.html
[+] NEW Total External links: 347
[+] NEW Total Internal links: 506
[+] NEW Total: 853
[+] NEW Total External links: 347
[+] NEW Total Internal links: 506
[+] NEW Total: 853
[+] NEW Total External links: 347
[+] NEW Total Internal links: 506
[+] NEW Total: 853
[INT] Internal link: http://www.umsl.edu/services/counser/About%20Counseling/privacy.html
[INT] Internal link: http://www.umsl.edu/services/counser/About%20Counseling/whattoexpect.html
[+] NEW Total External links: 347
[+] NEW Total Internal links: 508
[+] NEW Total: 855
[+] NEW Total External links: 356
[+] NEW Total Internal links: 508
[+] NEW Total: 864
[+] NEW Total External links: 356
[+] NEW Total Internal links: 508
[+] NEW Total: 864
[+] NEW Total Ext

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[+] NEW Total External links: 364
[+] NEW Total Internal links: 510
[+] NEW Total: 874


...

[+] NEW Total External links: 460
[+] NEW Total Internal links: 557
[+] NEW Total: 1017
[+] NEW Total External links: 460
[+] NEW Total Internal links: 557
[+] NEW Total: 1017


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[+] NEW Total External links: 460
[+] NEW Total Internal links: 557
[+] NEW Total: 1017
[+] NEW Total External links: 460
[+] NEW Total Internal links: 557
[+] NEW Total: 1017


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[+] NEW Total External links: 460
[+] NEW Total Internal links: 557
[+] NEW Total: 1017
[+] NEW Total External links: 460
[+] NEW Total Internal links: 557
[+] NEW Total: 1017
[+] NEW Total External links: 460
[+] NEW Total Internal links: 557
[+] NEW Total: 1017
[+] NEW Total External links: 460
[+] NEW Total Internal links: 557
[+] NEW Total: 1017
[+] NEW Total External links: 460
[+] NEW Total Internal links: 557
[+] NEW Total: 1017
[+] NEW Total External links: 460
[+] NEW Total Internal links: 557
[+] NEW Total: 1017
[+] NEW Total External links: 460
[+] NEW Total Internal links: 557
[+] NEW Total: 1017


In [ ]:
def removeAfterN(yourStr, nth, occurenceOf):
    return occurenceOf.join(yourStr.split(occurenceOf)[:nth])

In [ ]:
semifinal_immediate_urls = set()

for immediate_url in immediate_urls:
    stripped_url = removeAfterN(immediate_url,2,"http")
    
    
    semifinal_immediate_urls.add(stripped_url)
#     final_immediate_urls.add(rest)
#     print(rest)


    
for i in semifinal_immediate_urls:
    print(i)
total_count = len(semifinal_immediate_urls)
print(total_count)

In [ ]:
final_immediate_urls = set()
for semifinal_immediate_url in semifinal_immediate_urls:
    if "tel:" not in semifinal_immediate_url:
        final_immediate_urls.add(semifinal_immediate_url)


for i in final_immediate_urls:
    print(i)
total_count = len(final_immediate_urls)
print(total_count)       

In [ ]:

parent_folder = "Doctoral" #Change based on folder
name_of_folder = f'{base_url_text_domain}_Screenshots'
options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options,executable_path='chromedriver.exe') # #Local Path of chrome driver
if not os.path.exists(f'{parent_folder}\\{name_of_folder}'):
    os.makedirs(f'{parent_folder}\\{name_of_folder}')

In [ ]:
webpage_save = set()
webpage_save_count = 0
webpage_cannot_save = 0
for i in final_immediate_urls:
    if ' ' in i:
        continue
    try:
        code=urlopen(i).getcode()
        webpage_save.add(i)
        webpage_save_count = webpage_save_count+1
        print(f"{code} can_save - {i}")
    except (urllib.error.HTTPError,urllib.error.URLError) as e:
        print("discarded")
        webpage_cannot_save = webpage_cannot_save+1
        continue

print(f'Webpages save as HTML :{webpage_save_count}')
print(f'Webpages unable to save as HTML :{webpage_cannot_save}')
        

In [ ]:
pdf_count = 0
pdf_urls = set()
for webpage in webpage_save:
    try:
        if webpage[-4:] == ".pdf":
            url = webpage
            file_name = url.replace(base_url,'')
            file_name = file_name.replace('/','')
            if file_name == '':
                file_name = "Home"
            pdf_urls.add(webpage)
            print(webpage)
            r = requests.get(url, stream=True)
            with open(f'{parent_folder}\\{name_of_folder}\\{file_name}.pdf', 'wb') as fd:
                for chunk in r.iter_content(2000):
                    fd.write(chunk)
            pdf_count = pdf_count + 1
    except:
        print("caught")

print(f"Pdfs saved: {pdf_count}")

In [ ]:
final_immediate_urls = final_immediate_urls - pdf_urls
webpage_save = webpage_save - pdf_urls
web_save_count = len(webpage_save)
total_count = len(final_immediate_urls)
print(total_count) 

In [ ]:

#     try:
#         print(i)
        
#         print("added")
#     except urllib.error.HTTPError as e:
#         if e.code in (400,403):
#             print("discarded")
#             continue

In [ ]:
final_webpage_save_count = webpage_save_count - pdf_count
print(final_webpage_save_count)

In [ ]:
webpage_save_counter_loop = 0
for webpage in webpage_save:
    try:
        url = webpage
        file_name = url.replace(base_url,'')
        file_name = file_name.replace('/','')
        if file_name == '':
                file_name = "Home"
        urllib.request.urlretrieve (webpage, f"{parent_folder}\\{name_of_folder}\\{file_name}.html")
        webpage_save_counter_loop = webpage_save_counter_loop+1
        print(f'{webpage_save_counter_loop} of {final_webpage_save_count} - {webpage}')
    except:
        print("caught")

print("Saving Completed")

In [ ]:
count = 0
for i in final_immediate_urls:
    try:
        url = i
        file_name = url.replace(base_url,'')
        file_name = file_name.replace('/','')
        if file_name == '':
                file_name = "Home"
        count = count + 1
        print(f"{count} of {total_count}")
        print(f'Visiting {base_url_text}/{file_name}')
        print(f'...Taking a screenshot')
        driver.get(url)
        S = lambda X: driver.execute_script('return document.body.parentNode.scroll'+X)
        driver.set_window_size(S('Width'),S('Height')) # May need manual adjustment                                                                                                                
        driver.find_element_by_tag_name('body').screenshot(f'{parent_folder}\\{name_of_folder}\\{file_name}.png')
        print(f'Screenshot of {file_name} page taken! \n')
    except:
        print("caught")
#     print("Saving HTML")
#     pyautogui.hotkey('ctrl', 's')
#     time.sleep(1)
#     pyautogui.typewrite(f'{file_name}.html')
#     pyautogui.hotkey('enter')
driver.quit()
    
print(f'Task Completed! Files stored in the {parent_folder}\\{name_of_folder} Folder')